In [25]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import seaborn as sn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score, f1_score
cudnn.benchmark = True
plt.ion()   # interactive mode

In [26]:
# use GPU when available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Print GPU name
print('GPU: ', torch.cuda.get_device_name(device=device))
#number of usable CPU cores
core_count = os.cpu_count() #len(os.sched_getaffinity(0)) for server/linux system
print('CPU cores: ', core_count)
# current working directory
wd = os.getcwd()
print('Working directory: ', wd)

# start time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

GPU:  NVIDIA GeForce RTX 3060
CPU cores:  16
Working directory:  C:\Users\GCM\Desktop\GIT_REPOS\FML_playground\Final_few_scripts
14:36:09


In [8]:
# Loading Data
data = np.load(wd+'/nodup_data.npy')
labels = np.load(wd+'/nodup_labels.npy')

# reshape data
reshaped_data = data.reshape((300,300,3,data.shape[1]))
moveaxis_data = np.moveaxis(reshaped_data, source=[0, 1, 2, 3], destination=[2, 3, 1, 0])
labels = np.array(labels,dtype=int)

# label names
class_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only',
                'Do Not Enter','Crosswalk','Handicap Parking','No Parking']

# turn into tensors
# # and scale data
# to_tensor = transforms.ToTensor()
# to_tensor(reshaped_data[:,:,:,0])
data_te = torch.Tensor(moveaxis_data)
labels_te = torch.Tensor(labels)

# create additional images

# normalize parameters
stds = reshaped_data.std(axis=(0,1,3))
means = reshaped_data.mean(axis=(0,1,3))
normalize = torch.nn.Sequential(transforms.Normalize(means, stds)) # in shape of data_te


In [45]:
from torch.utils.data import Dataset

class CustomTensorDataset(Dataset):
    def __init__(self, dataset, transform_list=None):
        [data_X, data_y] = dataset
        X_tensor, y_tensor = torch.Tensor(data_X), torch.Tensor(data_y)
        tensors = (X_tensor, y_tensor)
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transforms = transform_list

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transforms:
          #for transform in self.transforms: 
          #  x = transform(x)
            x = self.transforms(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)


trainset = [moveaxis_data,labels]

transorm_comp = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=(0.75,1.25), 
                           contrast=(0,2), 
                           saturation=(0,2), 
                           hue=(-0.5,0.5)),
    transforms.GaussianBlur(kernel_size=(5, 9), 
                            sigma=(0.1, 5)),
    transforms.RandomPerspective(),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomAffine(degrees=(0, 360), 
                            translate=(0.1, 0.3), 
                            scale=(0.5, 0.75), 
                            shear=(0, 0.2, 0, 0.2)),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.Normalize(means, stds)
    ])

dataset_train = CustomTensorDataset(dataset=trainset, transform_list=transorm_comp)

dataset_dataloader = torch.utils.data.DataLoader(dataset_train, 
                                                 batch_size=4,
                                                 shuffle=True, 
                                                 num_workers=0)

In [9]:
TensorDataset(data_te,labels_te)

In [4]:
# transforms.ToTensor()

# transforms.functional.adjust_hue()
# transforms.functional.adjust_brightness()
# transforms.functional.adjust_contrast()
# transforms.functional.adjust_gamma()
# transforms.functional.adjust_saturation()
# transforms.functional.adjust_sharpness()
# transforms.functional.gaussian_blur()
# transforms.functional.hflip()
# transforms.functional.vflip()
# transforms.functional.rotate()

# transforms.functional.perspective()

In [5]:
# plt.imshow(torch.moveaxis(test_0, (0,1,2), (2,0,1)))

In [29]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=(0.75,1.25), 
                               contrast=(0,2), 
                               saturation=(0,2), 
                               hue=(-0.5,0.5)),
        transforms.GaussianBlur(kernel_size=(5, 9), 
                                sigma=(0.1, 5)),
        transforms.RandomPerspective(),
        transforms.RandomRotation(degrees=(0, 360)),
        transforms.RandomAffine(degrees=(0, 360), 
                                translate=(0.1, 0.3), 
                                scale=(0.5, 0.75), 
                                shear=(0, 0.2, 0, 0.2)),
        transforms.RandomAdjustSharpness(sharpness_factor=2),
        transforms.RandomEqualize(),
        transforms.Normalize(means, stds)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(means, stds)
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

NameError: name 'data_dir' is not defined

In [7]:
image_datasets = {'train': TensorDataset(x_train_te,t_train_te),
                 'val': TensorDataset(x_test_te,t_test_te)}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=core_count)
              for x in ['train', 'val']}

NameError: name 'x_train_te' is not defined